In [1]:
from typing import Union
import os
from os.path import join
import requests
import earthaccess
import pandas as pd
import datetime as dt
from datetime import date
from dateutil import parser
import geopandas
import geopandas as gpd
from shapely.geometry import MultiPolygon, Polygon, box
from shapely.geometry.polygon import orient
from sentinel_tiles import sentinel_tiles
from emit_tools.generate_CMR_daterange_string import generate_CMR_daterange_string
from emit_tools.search_EMIT_L2A_reflectance import search_EMIT_L2A_reflectance
from emit_tools.download_file import download_file
from emit_tools.download_scene import download_scene

In [2]:
geojson_filename = "11SPS.geojson"
start_date = "2024-05-01"
end_date = "2024-05-30"

In [3]:
# start_date = "2025-05-01"
# end_date = "2025-05-30"

In [4]:
# polygon = sentinel_tiles.footprint("11SPS")
# polygon.wkt

In [5]:
grid = sentinel_tiles.grid("11SPS", 60)
grid

{
  "dimensions": {
    "rows": 1830,
    "cols": 1830
  },
  "bbox": {
    "xmin": 600000.0,
    "ymin": 3590220.0,
    "xmax": 709800.0,
    "ymax": 3700020.0
  },
  "crs": "EPSG:32611",
  "resolution": {
    "cell_width": 60.0,
    "cell_height": -60.0
  }
}

In [6]:
df = search_EMIT_L2A_reflectance(
    start_date=start_date, 
    end_date=end_date, 
    target_area=grid
)

df

,date_UTC,product,filename,URL,cloud_percent,level,datetime_UTC,orbit,scene,collection,type,geometry
0,2024-05-21,L2A_RFL,EMIT_L2A_RFL_001_20240521T232716_2414216_002.nc,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,30,L2A,2024-05-21 23:27:16,2414216,2,1,data,"MULTIPOLYGON (((-115.98626 33.5353, -116.72288..."
1,2024-05-21,L2A_RFLUNCERT,EMIT_L2A_RFLUNCERT_001_20240521T232716_2414216...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,30,L2A,2024-05-21 23:27:16,2414216,2,1,data,"MULTIPOLYGON (((-115.98626 33.5353, -116.72288..."
2,2024-05-21,L2A_MASK,EMIT_L2A_MASK_001_20240521T232716_2414216_002.nc,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,30,L2A,2024-05-21 23:27:16,2414216,2,1,data,"MULTIPOLYGON (((-115.98626 33.5353, -116.72288..."
3,2024-05-21,L2A_RFL,EMIT_L2A_RFL_001_20240521T232716_2414216_002.png,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,30,L2A,2024-05-21 23:27:16,2414216,2,1,browse,"MULTIPOLYGON (((-115.98626 33.5353, -116.72288..."
0,2024-05-21,L2A_RFL,EMIT_L2A_RFL_001_20240521T232728_2414216_003.nc,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,20,L2A,2024-05-21 23:27:28,2414216,3,1,data,"MULTIPOLYGON (((-115.34624 34.05475, -116.1037..."
1,2024-05-21,L2A_RFLUNCERT,EMIT_L2A_RFLUNCERT_001_20240521T232728_2414216...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,20,L2A,2024-05-21 23:27:28,2414216,3,1,data,"MULTIPOLYGON (((-115.34624 34.05475, -116.1037..."
2,2024-05-21,L2A_MASK,EMIT_L2A_MASK_001_20240521T232728_2414216_003.nc,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,20,L2A,2024-05-21 23:27:28,2414216,3,1,data,"MULTIPOLYGON (((-115.34624 34.05475, -116.1037..."
3,2024-05-21,L2A_RFL,EMIT_L2A_RFL_001_20240521T232728_2414216_003.png,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,20,L2A,2024-05-21 23:27:28,2414216,3,1,browse,"MULTIPOLYGON (((-115.34624 34.05475, -116.1037..."
0,2024-05-21,L2A_RFL,EMIT_L2A_RFL_001_20240521T232740_2414216_004.nc,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,8,L2A,2024-05-21 23:27:40,2414216,4,1,data,"MULTIPOLYGON (((-114.70493 34.57001, -115.4730..."
1,2024-05-21,L2A_RFLUNCERT,EMIT_L2A_RFLUNCERT_001_20240521T232740_2414216...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,8,L2A,2024-05-21 23:27:40,2414216,4,1,data,"MULTIPOLYGON (((-114.70493 34.57001, -115.4730..."


In [7]:
URL = df.iloc[0].URL
URL

'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20240521T232716_2414216_002/EMIT_L2A_RFL_001_20240521T232716_2414216_002.nc'

In [8]:
filename_base = df.iloc[0].filename
filename_base

'EMIT_L2A_RFL_001_20240521T232716_2414216_002.nc'

In [9]:
filename = join("~/data/EMIT_download", filename_base)
filename

'~/data/EMIT_download/EMIT_L2A_RFL_001_20240521T232716_2414216_002.nc'

In [10]:
download_file(URL, filename)

[2024-10-25 14:50:59 INFO] File already downloaded: ~/data/EMIT_download/EMIT_L2A_RFL_001_20240521T232716_2414216_002.nc


'~/data/EMIT_download/EMIT_L2A_RFL_001_20240521T232716_2414216_002.nc'

In [13]:
directory = "~/data/EMIT_download/scene_directory"

In [14]:
download_scene(df, directory, 2414216, 2)

https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20240521T232716_2414216_002/EMIT_L2A_RFL_001_20240521T232716_2414216_002.nc
[2024-10-25 14:50:59 INFO] File already downloaded: ~/data/EMIT_download/scene_directory/EMIT_L2A_RFL_001_20240521T232716_2414216_002.nc
https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20240521T232716_2414216_002/EMIT_L2A_RFLUNCERT_001_20240521T232716_2414216_002.nc
[2024-10-25 14:50:59 INFO] File already downloaded: ~/data/EMIT_download/scene_directory/EMIT_L2A_RFLUNCERT_001_20240521T232716_2414216_002.nc
https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20240521T232716_2414216_002/EMIT_L2A_MASK_001_20240521T232716_2414216_002.nc
[2024-10-25 14:50:59 INFO] File already downloaded: ~/data/EMIT_download/scene_directory/EMIT_L2A_MASK_001_20240521T232716_2414216_002.nc
https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-public/EMITL2

['~/data/EMIT_download/scene_directory/EMIT_L2A_RFL_001_20240521T232716_2414216_002.nc',
 '~/data/EMIT_download/scene_directory/EMIT_L2A_RFLUNCERT_001_20240521T232716_2414216_002.nc',
 '~/data/EMIT_download/scene_directory/EMIT_L2A_MASK_001_20240521T232716_2414216_002.nc',
 '~/data/EMIT_download/scene_directory/EMIT_L2A_RFL_001_20240521T232716_2414216_002.png']